<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import time
import datetime
import subprocess

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence

from __future__ import absolute_import, division, print_function
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ls

drive/  runs/  sample_data/


In [0]:
def clean_str(text):
    text = text.lower()
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"that's", "that is ", text)
    text = re.sub(r"there's", "there is ", text)
    text = re.sub(r"it's", "it is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text.strip()

In [0]:
class2label = {'Other': 0,
               'Message-Topic(e1,e2)': 1, 'Message-Topic(e2,e1)': 2,
               'Product-Producer(e1,e2)': 3, 'Product-Producer(e2,e1)': 4,
               'Instrument-Agency(e1,e2)': 5, 'Instrument-Agency(e2,e1)': 6,
               'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8,
               'Cause-Effect(e1,e2)': 9, 'Cause-Effect(e2,e1)': 10,
               'Component-Whole(e1,e2)': 11, 'Component-Whole(e2,e1)': 12,
               'Entity-Origin(e1,e2)': 13, 'Entity-Origin(e2,e1)': 14,
               'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16,
               'Content-Container(e1,e2)': 17, 'Content-Container(e2,e1)': 18}

label2class = {0: 'Other',
               1: 'Message-Topic(e1,e2)', 2: 'Message-Topic(e2,e1)',
               3: 'Product-Producer(e1,e2)', 4: 'Product-Producer(e2,e1)',
               5: 'Instrument-Agency(e1,e2)', 6: 'Instrument-Agency(e2,e1)',
               7: 'Entity-Destination(e1,e2)', 8: 'Entity-Destination(e2,e1)',
               9: 'Cause-Effect(e1,e2)', 10: 'Cause-Effect(e2,e1)',
               11: 'Component-Whole(e1,e2)', 12: 'Component-Whole(e2,e1)',
               13: 'Entity-Origin(e1,e2)', 14: 'Entity-Origin(e2,e1)',
               15: 'Member-Collection(e1,e2)', 16: 'Member-Collection(e2,e1)',
               17: 'Content-Container(e1,e2)', 18: 'Content-Container(e2,e1)'}

In [0]:
def load_word2vec(word2vec_path, embedding_dim, vocab):
    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) * np.sqrt(2.0 / len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load word2vec file {0}".format(word2vec_path))
    with open(word2vec_path, "rb") as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            idx = vocab.vocabulary_.get(word)
            if idx != 0:
                initW[idx] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return initW


def load_glove(word2vec_path, embedding_dim, vocab):
    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) * np.sqrt(2.0 / len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load glove file {0}".format(word2vec_path))
    f = open(word2vec_path, 'r', encoding='utf8')
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        idx = vocab.vocabulary_.get(word)
        if idx != 0:
            initW[idx] = embedding
    return initW

In [0]:
def load_data_and_labels(path):
    data = []
    lines = [line.strip() for line in open(path)]
    max_sentence_length = 0
    for idx in range(0, len(lines), 4):
        id = lines[idx].split("\t")[0]
        relation = lines[idx + 1]

        sentence = lines[idx].split("\t")[1][1:-1]
        sentence = sentence.replace('<e1>', ' _e11_ ')
        sentence = sentence.replace('</e1>', ' _e12_ ')
        sentence = sentence.replace('<e2>', ' _e21_ ')
        sentence = sentence.replace('</e2>', ' _e22_ ')

        sentence = clean_str(sentence)
        tokens = nltk.word_tokenize(sentence)
        if max_sentence_length < len(tokens):
            max_sentence_length = len(tokens)
        e1 = tokens.index("e12") - 1
        e2 = tokens.index("e22") - 1
        sentence = " ".join(tokens)

        data.append([id, sentence, e1, e2, relation])

    print(path)
    print("max sentence length = {}\n".format(max_sentence_length))

    df = pd.DataFrame(data=data, columns=["id", "sentence", "e1", "e2", "relation"])

    pos1, pos2 = get_relative_position(df, 90)

    df['label'] = [class2label[r] for r in df['relation']]

    # Text Data
    x_text = df['sentence'].tolist()
    e1 = df['e1'].tolist()
    e2 = df['e2'].tolist()

    # Label Data
    y = df['label']
    labels_flat = y.values.ravel()
    labels_count = np.unique(labels_flat).shape[0]

    # convert class labels from scalars to one-hot vectors
    # 0  => [1 0 0 0 0 ... 0 0 0 0 0]
    # 1  => [0 1 0 0 0 ... 0 0 0 0 0]
    # ...
    # 18 => [0 0 0 0 0 ... 0 0 0 0 1]
    def dense_to_one_hot(labels_dense, num_classes):
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
        return labels_one_hot

    labels = dense_to_one_hot(labels_flat, labels_count)
    labels = labels.astype(np.uint8)

    return x_text, labels, e1, e2, pos1, pos2

In [0]:
def get_relative_position(df, max_sentence_length):
    # Position data
    pos1 = []
    pos2 = []
    for df_idx in range(len(df)):
        sentence = df.iloc[df_idx]['sentence']
        tokens = nltk.word_tokenize(sentence)
        e1 = df.iloc[df_idx]['e1']
        e2 = df.iloc[df_idx]['e2']

        p1 = ""
        p2 = ""
        for word_idx in range(len(tokens)):
            p1 += str((max_sentence_length - 1) + word_idx - e1) + " "
            p2 += str((max_sentence_length - 1) + word_idx - e2) + " "
        pos1.append(p1)
        pos2.append(p2)

    return pos1, pos2

In [0]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [11]:
text_train, y_train, train_e1, train_e2, train_pos1, train_pos2 = load_data_and_labels("drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT")

drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT
max sentence length = 89



In [12]:
text_test, y_test, test_e1, test_e2, test_pos1, test_pos2 = load_data_and_labels("drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT")

drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT
max sentence length = 68



In [0]:
import argparse
import sys

def parse_args():
    """
    Parse input arguments
    """
    parser = argparse.ArgumentParser()

    # Data loading params
    parser.add_argument("--train_path", default="drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT",
                        type=str, help="Path of train data")
    parser.add_argument("--test_path", default="drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT",
                        type=str, help="Path of test data")
    parser.add_argument("--max_sentence_length", default=90,
                        type=int, help="Max sentence length in data")

    # Model Hyper-parameters
    # Embeddings
    parser.add_argument("--embeddings", default=None,
                        type=str, help="Embeddings {'word2vec', 'glove100', 'glove300', 'elmo'}")
    parser.add_argument("--embedding_size", default=300,
                        type=int, help="Dimensionality of word embedding (default: 300)")
    parser.add_argument("--pos_embedding_size", default=50,
                        type=int, help="Dimensionality of relative position embedding (default: 50)")
    parser.add_argument("--emb_dropout_keep_prob", default=0.7,
                        type=float, help="Dropout keep probability of embedding layer (default: 0.7)")
    # RNN
    parser.add_argument("--hidden_size", default=300,
                        type=int, help="Dimensionality of RNN hidden (default: 300)")
    parser.add_argument("--rnn_dropout_keep_prob", default=0.7,
                        type=float, help="Dropout keep probability of RNN (default: 0.7)")
    # Attention
    parser.add_argument("--num_heads", default=4,
                        type=int, help="Number of heads in multi-head attention (default: 4)")
    parser.add_argument("--attention_size", default=50,
                        type=int, help="Dimensionality of attention (default: 50)")
    # Misc
    parser.add_argument("--desc", default="",
                        type=str, help="Description for model")
    parser.add_argument("--dropout_keep_prob", default=0.5,
                        type=float, help="Dropout keep probability of output layer (default: 0.5)")
    parser.add_argument("--l2_reg_lambda", default=1e-5,
                        type=float, help="L2 regularization lambda (default: 1e-5)")

    # Training parameters
    parser.add_argument("--batch_size", default=20,
                        type=int, help="Batch Size (default: 20)")
    parser.add_argument("--num_epochs", default=100,
                        type=int, help="Number of training epochs (Default: 100)")
    parser.add_argument("--display_every", default=10,
                        type=int, help="Number of iterations to display training information")
    parser.add_argument("--evaluate_every", default=100,
                        type=int, help="Evaluate model on dev set after this many steps (default: 100)")
    parser.add_argument("--num_checkpoints", default=5,
                        type=int, help="Number of checkpoints to store (default: 5)")
    parser.add_argument("--learning_rate", default=1.0,
                        type=float, help="Which learning rate to start with (Default: 1.0)")
    parser.add_argument("--decay_rate", default=0.9,
                        type=float, help="Decay rate for learning rate (Default: 0.9)")

    # Misc Parameters
    parser.add_argument("--allow_soft_placement", default=True,
                        type=bool, help="Allow device soft device placement")
    parser.add_argument("--log_device_placement", default=False,
                        type=bool, help="Log placement of ops on devices")
    parser.add_argument("--gpu_allow_growth", default=True,
                        type=bool, help="Allow gpu memory growth")

    # Visualization Parameters
    parser.add_argument("--checkpoint_dir", default=None,
                        type=str, help="Visualize this checkpoint")

    if len(sys.argv) == 0:
        parser.print_help()
        sys.exit(1)

    print("")
    args = parser.parse_known_args()[0]
    for arg in vars(args):
        print("{}={}".format(arg.upper(), getattr(args, arg)))
    print("")

    return args

In [14]:
FLAGS = parse_args()


TRAIN_PATH=drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT
TEST_PATH=drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT
MAX_SENTENCE_LENGTH=90
EMBEDDINGS=None
EMBEDDING_SIZE=300
POS_EMBEDDING_SIZE=50
EMB_DROPOUT_KEEP_PROB=0.7
HIDDEN_SIZE=300
RNN_DROPOUT_KEEP_PROB=0.7
NUM_HEADS=4
ATTENTION_SIZE=50
DESC=
DROPOUT_KEEP_PROB=0.5
L2_REG_LAMBDA=1e-05
BATCH_SIZE=20
NUM_EPOCHS=100
DISPLAY_EVERY=10
EVALUATE_EVERY=100
NUM_CHECKPOINTS=5
LEARNING_RATE=1.0
DECAY_RATE=0.9
ALLOW_SOFT_PLACEMENT=True
LOG_DEVICE_PLACEMENT=False
GPU_ALLOW_GROWTH=True
CHECKPOINT_DIR=None



In [15]:
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(90)
vocab_processor.fit(text_train + text_test)
x_train = np.array(list(vocab_processor.transform(text_train)))
x_test = np.array(list(vocab_processor.transform(text_test)))
text_train = np.array(text_train)
text_test = np.array(text_test)
print("\nText Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("x_train = {0}".format(x_train.shape))
print("y_train = {0}".format(y_train.shape))
print("x_test = {0}".format(x_test.shape))
print("y_test = {0}".format(y_test.shape))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.

Text Vocabulary Size: 22384
x_train = (8000, 90)
y_train = (8000, 19)
x_test = (2717, 90)
y_test = (2717, 19)


In [16]:
pos_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(FLAGS.max_sentence_length)
pos_vocab_processor.fit(train_pos1 + train_pos2 + test_pos1 + test_pos2)
train_p1 = np.array(list(pos_vocab_processor.transform(train_pos1)))
train_p2 = np.array(list(pos_vocab_processor.transform(train_pos2)))
test_p1 = np.array(list(pos_vocab_processor.transform(test_pos1)))
test_p2 = np.array(list(pos_vocab_processor.transform(test_pos2)))
print("\nPosition Vocabulary Size: {:d}".format(len(pos_vocab_processor.vocabulary_)))
print("train_p1 = {0}".format(train_p1.shape))
print("test_p1 = {0}".format(test_p1.shape))
print("")


Position Vocabulary Size: 162
train_p1 = (8000, 90)
test_p1 = (2717, 90)



In [0]:
def initializer():
    return tf.keras.initializers.glorot_normal()

In [0]:
def load_word2vec(word2vec_path, embedding_dim, vocab):
    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) * np.sqrt(2.0 / len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load word2vec file {0}".format(word2vec_path))
    with open(word2vec_path, "rb") as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            idx = vocab.vocabulary_.get(word)
            if idx != 0:
                initW[idx] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return initW

In [0]:
def load_glove(word2vec_path, embedding_dim, vocab):
    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) * np.sqrt(2.0 / len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load glove file {0}".format(word2vec_path))
    f = open(word2vec_path, 'r', encoding='utf8')
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        idx = vocab.vocabulary_.get(word)
        if idx != 0:
            initW[idx] = embedding
    return initW

In [0]:
def attention(inputs, e1, e2, p1, p2, attention_size):
    # inputs = (batch, seq_len, hidden)
    # e1, e2 = (batch, seq_len)
    # p1, p2 = (batch, seq_len, dist_emb_size)
    # attention_size = scalar(int)
    def extract_entity(x, e):
        e_idx = tf.concat([tf.expand_dims(tf.range(tf.shape(e)[0]), axis=-1), tf.expand_dims(e, axis=-1)], axis=-1)
        return tf.gather_nd(x, e_idx)  # (batch, hidden)
    seq_len = tf.shape(inputs)[1]  # fixed at run-time
    hidden_size = inputs.shape[2].value  # fixed at compile-time
    latent_size = hidden_size

    # Latent Relation Variable based on Entities
    e1_h = extract_entity(inputs, e1)  # (batch, hidden)
    e2_h = extract_entity(inputs, e2)  # (batch, hidden)
    e1_type, e2_type, e1_alphas, e2_alphas = latent_type_attention(e1_h, e2_h,
                                                                   num_type=3,
                                                                   latent_size=latent_size)  # (batch, hidden)
    e1_h = tf.concat([e1_h, e1_type], axis=-1)  # (batch, hidden+latent)
    e2_h = tf.concat([e2_h, e2_type], axis=-1)  # (batch, hidden+latent)

    # v*tanh(W*[h;p1;p2]+W*[e1;e2]) 85.18%? 84.83% 84.55%
    e_h = tf.layers.dense(tf.concat([e1_h, e2_h], -1), attention_size, use_bias=False, kernel_initializer=initializer())
    e_h = tf.reshape(tf.tile(e_h, [1, seq_len]), [-1, seq_len, attention_size])
    v = tf.layers.dense(tf.concat([inputs, p1, p2], axis=-1), attention_size, use_bias=False, kernel_initializer=initializer())
    v = tf.tanh(tf.add(v, e_h))

    u_omega = tf.get_variable("u_omega", [attention_size], initializer=initializer())
    vu = tf.tensordot(v, u_omega, axes=1, name='vu')  # (batch, seq_len)
    alphas = tf.nn.softmax(vu, name='alphas')  # (batch, seq_len)

    # v*tanh(W*[h;p1;p2;e1;e2]) 85.18% 84.41%
    # e1_h = tf.reshape(tf.tile(e1_h, [1, seq_len]), [-1, seq_len, hidden_size+latent_size])
    # e2_h = tf.reshape(tf.tile(e2_h, [1, seq_len]), [-1, seq_len, hidden_size+latent_size])
    # v = tf.concat([inputs, p1, p2, e1_h, e2_h], axis=-1)
    # v = tf.layers.dense(v, attention_size, activation=tf.tanh, kernel_initializer=initializer())
    #
    # u_omega = tf.get_variable("u_omega", [attention_size], initializer=initializer())
    # vu = tf.tensordot(v, u_omega, axes=1, name='vu')  # (batch, seq_len)
    # alphas = tf.nn.softmax(vu, name='alphas')  # (batch, seq_len)

    # output
    output = tf.reduce_sum(inputs * tf.expand_dims(alphas, -1), 1)  # (batch, hidden)

    return output, alphas, e1_alphas, e2_alphas

In [0]:
def latent_type_attention(e1, e2, num_type, latent_size):
    # Latent Entity Type Vectors
    latent_type = tf.get_variable("latent_type", shape=[num_type, latent_size], initializer=initializer())

    # e1_h = tf.layers.dense(e1, latent_size, kernel_initializer=initializer())
    # e2_h = tf.layers.dense(e2, latent_size, kernel_initializer=initializer())

    e1_sim = tf.matmul(e1, tf.transpose(latent_type))  # (batch, num_type)
    e1_alphas = tf.nn.softmax(e1_sim, name='e1_alphas')  # (batch, num_type)
    e1_type = tf.matmul(e1_alphas, latent_type, name='e1_type')  # (batch, hidden)

    e2_sim = tf.matmul(e2, tf.transpose(latent_type))  # (batch, num_type)
    e2_alphas = tf.nn.softmax(e2_sim, name='e2_alphas')  # (batch, num_type)
    e2_type = tf.matmul(e2_alphas, latent_type, name='e2_type')  # (batch, hidden)

    return e1_type, e2_type, e1_alphas, e2_alphas

In [0]:
def multihead_attention(queries, keys, num_units, num_heads,
                        dropout_rate=0, scope="multihead_attention", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        # Linear projections
        Q = tf.layers.dense(queries, num_units, kernel_initializer=initializer())  # (N, T_q, C)
        K = tf.layers.dense(keys, num_units, kernel_initializer=initializer())  # (N, T_k, C)
        V = tf.layers.dense(keys, num_units, kernel_initializer=initializer())  # (N, T_k, C)

        # Split and concat
        Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0)  # (h*N, T_q, C/h)
        K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0)  # (h*N, T_k, C/h)
        V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0)  # (h*N, T_k, C/h)

        # Multiplication
        outputs = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1]))  # (h*N, T_q, T_k)

        # Scale
        outputs /= K_.get_shape().as_list()[-1] ** 0.5

        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1)))  # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1])  # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1])  # (h*N, T_q, T_k)

        paddings = tf.ones_like(outputs) * (-2 ** 32 + 1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs)  # (h*N, T_q, T_k)

        # Activation
        alphas = tf.nn.softmax(outputs)  # (h*N, T_q, T_k)

        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(queries, axis=-1)))  # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1])  # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]])  # (h*N, T_q, T_k)
        alphas *= query_masks  # broadcasting. (N, T_q, C)

        # Dropouts
        alphas = tf.layers.dropout(alphas, rate=dropout_rate, training=tf.convert_to_tensor(True))

        # Weighted sum
        outputs = tf.matmul(alphas, V_)  # ( h*N, T_q, C/h)
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2)  # (N, T_q, C)

        # Linear
        outputs = tf.layers.dense(outputs, num_units, activation=tf.nn.relu, kernel_initializer=initializer())

        # Residual connection
        outputs += queries
        # Normalize
        outputs = layer_norm(outputs)  # (N, T_q, C)

    return outputs, alphas

In [0]:
def layer_norm(inputs, epsilon=1e-8, scope="layer_norm", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]

        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta = tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ((variance + epsilon) ** (.5))
        outputs = gamma * normalized + beta
    return outputs

In [0]:
class Logger:
    def __init__(self, out_dir):
        self.log_dir = os.path.abspath(os.path.join(out_dir, "logs"))
        os.makedirs(self.log_dir)
        self.log_path = os.path.abspath(os.path.join(self.log_dir, "logs.txt"))
        self.log_file = open(self.log_path, "w")

        self.print_hyperparameters()

        self.best_f1 = 0.0

    def print_hyperparameters(self):
        self.log_file.write("\n================ Hyper-parameters ================\n\n")
        for arg in vars(FLAGS):
            self.log_file.write("{}={}\n".format(arg.upper(), getattr(FLAGS, arg)))
        self.log_file.write("\n==================================================\n\n")

    def logging_train(self, step, loss, accuracy):
        time_str = datetime.datetime.now().isoformat()
        log = "{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy)
        self.log_file.write(log+"\n")
        print(log)

    def logging_eval(self, step, loss, accuracy, predictions):
        self.log_file.write("\nEvaluation:\n")
        # loss & acc
        time_str = datetime.datetime.now().isoformat()
        log = "{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy)
        self.log_file.write(log + "\n")
        print(log)
#         print("=========Prediction_Path============")
#         print(self.log_dir)
        # f1-score
        prediction_path = os.path.abspath(os.path.join(self.log_dir, "predictions.txt"))[9:]
#         print(prediction_path)
        prediction_file = open(prediction_path, 'w')
        for i in range(len(predictions)):
            prediction_file.write("{}\t{}\n".format(i, label2class[predictions[i]]))
        prediction_file.close()
        perl_path = "drive/My Drive/7374Assignment4/SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/semeval2010_task8_scorer-v1.2.pl"
#         os.path.join(os.path.curdir,
#                                  "SemEval2010_task8_all_data",
#                                  "SemEval2010_task8_scorer-v1.2",
#                                  "semeval2010_task8_scorer-v1.2.pl")
        target_path = "drive/My Drive/7374Assignment4/resource/target.txt"
        process = subprocess.Popen(["perl", perl_path, prediction_path, target_path], stdout=subprocess.PIPE)
#         print("===========logger===========")
#         print(perl_path)
#         print(prediction_path)
#         print(target_path)
#         print(process)
        str_parse = str(process.communicate()[0]).split("\\n")[-2]
        idx = str_parse.find('%')
        f1_score = float(str_parse[idx-5:idx])

        self.best_f1 = max(self.best_f1, f1_score)
        f1_log = "<<< (9+1)-WAY EVALUATION TAKING DIRECTIONALITY INTO ACCOUNT -- OFFICIAL >>>:\n" \
                 "macro-averaged F1-score = {:g}%, Best = {:g}%\n".format(f1_score, self.best_f1)
        self.log_file.write(f1_log + "\n")
        print(f1_log)

In [0]:
class EntityAttentionLSTM:
    def __init__(self, sequence_length, num_classes,
                 vocab_size, embedding_size, pos_vocab_size, pos_embedding_size,
                 hidden_size, num_heads, attention_size,
                 use_elmo=False, l2_reg_lambda=0.0):
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, shape=[None, num_classes], name='input_y')
        self.input_text = tf.placeholder(tf.string, shape=[None, ], name='input_text')
        self.input_e1 = tf.placeholder(tf.int32, shape=[None, ], name='input_e1')
        self.input_e2 = tf.placeholder(tf.int32, shape=[None, ], name='input_e2')
        self.input_p1 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p1')
        self.input_p2 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p2')
        self.emb_dropout_keep_prob = tf.placeholder(tf.float32, name='emb_dropout_keep_prob')
        self.rnn_dropout_keep_prob = tf.placeholder(tf.float32, name='rnn_dropout_keep_prob')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')

        if use_elmo:
            # Contextual Embedding Layer
            with tf.variable_scope("elmo-embeddings"):
                elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
                self.embedded_chars = elmo_model(self.input_text, signature="default", as_dict=True)["elmo"]
        else:
            # Word Embedding Layer
            with tf.device('/cpu:0'), tf.variable_scope("word-embeddings"):
                self.W_text = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -0.25, 0.25), name="W_text")
                self.embedded_chars = tf.nn.embedding_lookup(self.W_text, self.input_x)

        # Position Embedding Layer
        with tf.device('/cpu:0'), tf.variable_scope("position-embeddings"):
            self.W_pos = tf.get_variable("W_pos", [pos_vocab_size, pos_embedding_size], initializer=initializer())
            self.p1 = tf.nn.embedding_lookup(self.W_pos, self.input_p1)[:, :tf.shape(self.embedded_chars)[1]]
            self.p2 = tf.nn.embedding_lookup(self.W_pos, self.input_p2)[:, :tf.shape(self.embedded_chars)[1]]

        # Dropout for Word Embedding
        with tf.variable_scope('dropout-embeddings'):
            self.embedded_chars = tf.nn.dropout(self.embedded_chars,  self.emb_dropout_keep_prob)

        # Self Attention
        with tf.variable_scope("self-attention"):
            self.self_attn, self.self_alphas = multihead_attention(self.embedded_chars, self.embedded_chars,
                                                                   num_units=embedding_size, num_heads=num_heads)

        # Bidirectional LSTM
        with tf.variable_scope("bi-lstm"):
            _fw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size, initializer=initializer())
            fw_cell = tf.nn.rnn_cell.DropoutWrapper(_fw_cell, self.rnn_dropout_keep_prob)
            _bw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size, initializer=initializer())
            bw_cell = tf.nn.rnn_cell.DropoutWrapper(_bw_cell, self.rnn_dropout_keep_prob)
            self.rnn_outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell,
                                                                  cell_bw=bw_cell,
                                                                  inputs=self.self_attn,
                                                                  sequence_length=self._length(self.input_x),
                                                                  dtype=tf.float32)
            self.rnn_outputs = tf.concat(self.rnn_outputs, axis=-1)

        # Attention
        with tf.variable_scope('attention'):
            self.attn, self.alphas, self.e1_alphas, self.e2_alphas = attention(self.rnn_outputs,
                                                                               self.input_e1, self.input_e2,
                                                                               self.p1, self.p2,
                                                                               attention_size=attention_size)

        # Dropout
        with tf.variable_scope('dropout'):
            self.h_drop = tf.nn.dropout(self.attn, self.dropout_keep_prob)

        # Fully connected layer
        with tf.variable_scope('output'):
            self.logits = tf.layers.dense(self.h_drop, num_classes, kernel_initializer=initializer())
            self.predictions = tf.argmax(self.logits, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.variable_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.input_y)
            self.l2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * self.l2

        # Accuracy
        with tf.variable_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name="accuracy")

    # Length of the sequence data
    @staticmethod
    def _length(seq):
        relevant = tf.sign(tf.abs(seq))
        length = tf.reduce_sum(relevant, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length

In [26]:
with tf.Graph().as_default():
  session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
  session_conf.gpu_options.allow_growth = FLAGS.gpu_allow_growth
  sess = tf.Session(config=session_conf)
  with sess.as_default():
    model = EntityAttentionLSTM(
        sequence_length=x_train.shape[1],
        num_classes=y_train.shape[1],
        vocab_size=len(vocab_processor.vocabulary_),
        embedding_size=FLAGS.embedding_size,
        pos_vocab_size=len(pos_vocab_processor.vocabulary_),
        pos_embedding_size=FLAGS.pos_embedding_size,
        hidden_size=FLAGS.hidden_size,
        num_heads=FLAGS.num_heads,
        attention_size=FLAGS.attention_size,
        use_elmo=(FLAGS.embeddings == 'elmo'),
        l2_reg_lambda=FLAGS.l2_reg_lambda)

    # Define Training procedure
    global_step = tf.Variable(0, name="global_step", trainable=False)
    optimizer = tf.train.AdadeltaOptimizer(FLAGS.learning_rate, FLAGS.decay_rate, 1e-6)
    gvs = optimizer.compute_gradients(model.loss)
    capped_gvs = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

    # Output directory for models and summaries
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
    print("\nWriting to {}\n".format(out_dir))

    # Logger
    logger = Logger(out_dir)

    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", model.loss)
    acc_summary = tf.summary.scalar("accuracy", model.accuracy)

    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, acc_summary])
    train_summary_dir = os.path.join(out_dir, "summaries", "train")
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

    # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
    checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))[9:]
    print(checkpoint_dir)
    checkpoint_prefix = os.path.join(checkpoint_dir, "model")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

    # Write vocabulary
    vocab_processor.save(os.path.join(out_dir, "vocab"))
    pos_vocab_processor.save(os.path.join(out_dir, "pos_vocab"))

    # Initialize all variables
    sess.run(tf.global_variables_initializer())

    if FLAGS.embeddings == "word2vec":
        pretrain_W = load_word2vec('resource/GoogleNews-vectors-negative300.bin', FLAGS.embedding_size, vocab_processor)
        sess.run(model.W_text.assign(pretrain_W))
        print("Success to load pre-trained word2vec model!\n")
    elif FLAGS.embeddings == "glove100":
        pretrain_W = load_glove('drive/My Drive/7374Assignment4/resource/glove.6B.300d.txt', FLAGS.embedding_size, vocab_processor)
        sess.run(model.W_text.assign(pretrain_W))
        print("Success to load pre-trained glove100 model!\n")
    elif FLAGS.embeddings == "glove300":
        pretrain_W = load_glove('drive/My Drive/7374Assignment4/resource/glove.6B.300d.txt', FLAGS.embedding_size, vocab_processor)
        sess.run(model.W_text.assign(pretrain_W))
        print("Success to load pre-trained glove300 model!\n")
    # Generate batches
    train_batches = batch_iter(list(zip(x_train, y_train, text_train,
                                                     train_e1, train_e2, train_p1, train_p2)),
                                            FLAGS.batch_size, FLAGS.num_epochs)
    # Training loop. For each batch...
    best_f1 = 0.0  # For save checkpoint(model)
    for train_batch in train_batches:
        train_bx, train_by, train_btxt, train_be1, train_be2, train_bp1, train_bp2 = zip(*train_batch)
        feed_dict = {
            model.input_x: train_bx,
            model.input_y: train_by,
            model.input_text: train_btxt,
            model.input_e1: train_be1,
            model.input_e2: train_be2,
            model.input_p1: train_bp1,
            model.input_p2: train_bp2,
            model.emb_dropout_keep_prob: FLAGS.emb_dropout_keep_prob,
            model.rnn_dropout_keep_prob: FLAGS.rnn_dropout_keep_prob,
            model.dropout_keep_prob: FLAGS.dropout_keep_prob
        }
        _, step, summaries, loss, accuracy = sess.run(
            [train_op, global_step, train_summary_op, model.loss, model.accuracy], feed_dict)
        train_summary_writer.add_summary(summaries, step)

        # Training log display
        if step % FLAGS.display_every == 0:
            logger.logging_train(step, loss, accuracy)

        # Evaluation
        if step % FLAGS.evaluate_every == 0:
            print("\nEvaluation:")
            # Generate batches
            test_batches = batch_iter(list(zip(x_test, y_test, text_test,
                                                            test_e1, test_e2, test_p1, test_p2)),
                                                   FLAGS.batch_size, 1, shuffle=False)
            # Training loop. For each batch...
            losses = 0.0
            accuracy = 0.0
            predictions = []
            iter_cnt = 0
            for test_batch in test_batches:
                test_bx, test_by, test_btxt, test_be1, test_be2, test_bp1, test_bp2 = zip(*test_batch)
                feed_dict = {
                    model.input_x: test_bx,
                    model.input_y: test_by,
                    model.input_text: test_btxt,
                    model.input_e1: test_be1,
                    model.input_e2: test_be2,
                    model.input_p1: test_bp1,
                    model.input_p2: test_bp2,
                    model.emb_dropout_keep_prob: 1.0,
                    model.rnn_dropout_keep_prob: 1.0,
                    model.dropout_keep_prob: 1.0
                }
                loss, acc, pred = sess.run(
                    [model.loss, model.accuracy, model.predictions], feed_dict)
                losses += loss
                accuracy += acc
                predictions += pred.tolist()
                iter_cnt += 1
            losses /= iter_cnt
            accuracy /= iter_cnt
            predictions = np.array(predictions, dtype='int')

            logger.logging_eval(step, loss, accuracy, predictions)

            # Model checkpoint
            if best_f1 < logger.best_f1:
                best_f1 = logger.best_f1
                path = saver.save(sess, checkpoint_prefix+"-{:.3g}".format(best_f1), global_step=step)
                print("Saved model checkpoint to {}\n".format(path))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.

Writing to /content/runs/1554245404

runs/1554245404/checkpoints
2019-04-02T22:50:08.449744: step 10, loss 3.60908, acc 0.15
2019-04-02T22:50:10.724233: step 20, loss 3.27118, acc 0.3
2019-04-02T22:50:12.973487: step 30, loss 3.41416, acc 0.2
2019-04-02T22:50:15.134916: step 40, loss 3.37067, a

In [0]:
with tf.Graph().as_default():
  session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
  session_conf.gpu_options.allow_growth = FLAGS.gpu_allow_growth
  sess = tf.Session(config=session_conf)
  with sess.as_default():
    model = EntityAttentionLSTM(
        sequence_length=x_train.shape[1],
        num_classes=y_train.shape[1],
        vocab_size=len(vocab_processor.vocabulary_),
        embedding_size=FLAGS.embedding_size,
        pos_vocab_size=len(pos_vocab_processor.vocabulary_),
        pos_embedding_size=FLAGS.pos_embedding_size,
        hidden_size=FLAGS.hidden_size,
        num_heads=FLAGS.num_heads,
        attention_size=FLAGS.attention_size,
        use_elmo=(FLAGS.embeddings == 'elmo'),
        l2_reg_lambda=FLAGS.l2_reg_lambda)

    # Define Training procedure
    global_step = tf.Variable(0, name="global_step", trainable=False)
    optimizer = tf.train.AdadeltaOptimizer(FLAGS.learning_rate, FLAGS.decay_rate, 1e-6)
    gvs = optimizer.compute_gradients(model.loss)
    capped_gvs = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

    # Output directory for models and summaries
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
    print("\nWriting to {}\n".format(out_dir))

    # Logger
    logger = Logger(out_dir)

    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", model.loss)
    acc_summary = tf.summary.scalar("accuracy", model.accuracy)

    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, acc_summary])
    train_summary_dir = os.path.join(out_dir, "summaries", "train")
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

    # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
    checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))[9:]
    print(checkpoint_dir)
    checkpoint_prefix = os.path.join(checkpoint_dir, "model")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

    # Write vocabulary
    vocab_processor.save(os.path.join(out_dir, "vocab"))
    pos_vocab_processor.save(os.path.join(out_dir, "pos_vocab"))

    # Initialize all variables
    sess.run(tf.global_variables_initializer())

#     if FLAGS.embeddings == "word2vec":
#         pretrain_W = load_word2vec('resource/GoogleNews-vectors-negative300.bin', FLAGS.embedding_size, vocab_processor)
#         sess.run(model.W_text.assign(pretrain_W))
#         print("Success to load pre-trained word2vec model!\n")
#     elif FLAGS.embeddings == "glove100":
#         pretrain_W = load_glove('drive/My Drive/7374Assignment4/resource/glove.6B.300d.txt', FLAGS.embedding_size, vocab_processor)
#         sess.run(model.W_text.assign(pretrain_W))
#         print("Success to load pre-trained glove100 model!\n")
#     elif FLAGS.embeddings == "glove300":
#         pretrain_W = load_glove('drive/My Drive/7374Assignment4/resource/glove.6B.300d.txt', FLAGS.embedding_size, vocab_processor)
#         sess.run(model.W_text.assign(pretrain_W))
#         print("Success to load pre-trained glove300 model!\n")
    pretrain_W = load_glove('drive/My Drive/7374Assignment4/resource/glove.6B.300d.txt', FLAGS.embedding_size, vocab_processor)
    sess.run(model.W_text.assign(pretrain_W))
    print("Success to load pre-trained glove300 model!\n")
    # Generate batches
    train_batches = batch_iter(list(zip(x_train, y_train, text_train,
                                                     train_e1, train_e2, train_p1, train_p2)),
                                            FLAGS.batch_size, FLAGS.num_epochs)
    # Training loop. For each batch...
    best_f1 = 0.0  # For save checkpoint(model)
    for train_batch in train_batches:
        train_bx, train_by, train_btxt, train_be1, train_be2, train_bp1, train_bp2 = zip(*train_batch)
        feed_dict = {
            model.input_x: train_bx,
            model.input_y: train_by,
            model.input_text: train_btxt,
            model.input_e1: train_be1,
            model.input_e2: train_be2,
            model.input_p1: train_bp1,
            model.input_p2: train_bp2,
            model.emb_dropout_keep_prob: FLAGS.emb_dropout_keep_prob,
            model.rnn_dropout_keep_prob: FLAGS.rnn_dropout_keep_prob,
            model.dropout_keep_prob: FLAGS.dropout_keep_prob
        }
        _, step, summaries, loss, accuracy = sess.run(
            [train_op, global_step, train_summary_op, model.loss, model.accuracy], feed_dict)
        train_summary_writer.add_summary(summaries, step)

        # Training log display
        if step % FLAGS.display_every == 0:
            logger.logging_train(step, loss, accuracy)

        # Evaluation
        if step % FLAGS.evaluate_every == 0:
            print("\nEvaluation:")
            # Generate batches
            test_batches = batch_iter(list(zip(x_test, y_test, text_test,
                                                            test_e1, test_e2, test_p1, test_p2)),
                                                   FLAGS.batch_size, 1, shuffle=False)
            # Training loop. For each batch...
            losses = 0.0
            accuracy = 0.0
            predictions = []
            iter_cnt = 0
            for test_batch in test_batches:
                test_bx, test_by, test_btxt, test_be1, test_be2, test_bp1, test_bp2 = zip(*test_batch)
                feed_dict = {
                    model.input_x: test_bx,
                    model.input_y: test_by,
                    model.input_text: test_btxt,
                    model.input_e1: test_be1,
                    model.input_e2: test_be2,
                    model.input_p1: test_bp1,
                    model.input_p2: test_bp2,
                    model.emb_dropout_keep_prob: 1.0,
                    model.rnn_dropout_keep_prob: 1.0,
                    model.dropout_keep_prob: 1.0
                }
                loss, acc, pred = sess.run(
                    [model.loss, model.accuracy, model.predictions], feed_dict)
                losses += loss
                accuracy += acc
                predictions += pred.tolist()
                iter_cnt += 1
            losses /= iter_cnt
            accuracy /= iter_cnt
            predictions = np.array(predictions, dtype='int')

            logger.logging_eval(step, loss, accuracy, predictions)

            # Model checkpoint
            if best_f1 < logger.best_f1:
                best_f1 = logger.best_f1
                path = saver.save(sess, checkpoint_prefix+"-{:.3g}".format(best_f1), global_step=step)
                print("Saved model checkpoint to {}\n".format(path))


Writing to /content/runs/1554259180

runs/1554259180/checkpoints
Load glove file drive/My Drive/7374Assignment4/resource/glove.6B.300d.txt
Success to load pre-trained glove300 model!

2019-04-03T02:40:24.122443: step 10, loss 7.39542, acc 0.15
2019-04-03T02:40:26.287588: step 20, loss 7.33323, acc 0.05
2019-04-03T02:40:28.786198: step 30, loss 7.4801, acc 0.1
2019-04-03T02:40:30.995681: step 40, loss 6.62555, acc 0.4
2019-04-03T02:40:33.382643: step 50, loss 6.46004, acc 0.4
2019-04-03T02:40:35.687315: step 60, loss 6.50808, acc 0.4
2019-04-03T02:40:38.069619: step 70, loss 6.90266, acc 0.3
2019-04-03T02:40:40.404613: step 80, loss 6.22348, acc 0.4
2019-04-03T02:40:42.802134: step 90, loss 5.80232, acc 0.6
2019-04-03T02:40:45.192250: step 100, loss 6.10879, acc 0.45

Evaluation:
2019-04-03T02:40:55.043155: step 100, loss 6.60494, acc 0.586786
<<< (9+1)-WAY EVALUATION TAKING DIRECTIONALITY INTO ACCOUNT -- OFFICIAL >>>:
macro-averaged F1-score = 59.16%, Best = 59.16%

Saved model checkp